## 04 Valuation Tool

In [142]:
#from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np
import io
import requests

In [143]:
# raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
# data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
# target = raw_df.values[1::2, 2]

data_url = "http://lib.stat.cmu.edu/datasets/boston"
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']

try:
    # Fetch the raw text content
    response = requests.get(data_url)
    response.raise_for_status() # Raise an exception for bad status codes
    raw_content = response.text
except requests.exceptions.RequestException as e:
    print(f"Error fetching data from {data_url}: {e}")
    print("Please check your internet connection or if the URL is still valid.")
    exit()

# Split the content into lines
lines = raw_content.split('\n')

# The actual data starts after a preamble (usually line 22, 0-indexed)
# Let's find the start of the data by looking for lines that are not part of the description
data_start_line = 0
for i, line in enumerate(lines):
    if line.strip().startswith('1.'): # Heuristic: data usually starts with "1." for the first row
        data_start_line = i
        break
    elif i > 25: # Fallback: if not found by line 25, assume data starts after standard preamble
        data_start_line = 22
        break
# If the loop finishes without finding "1.", data_start_line will be 0.
# A more robust check might be to look for empty lines after the description.

# Process the lines to combine two physical lines into one logical row
processed_lines = []
temp_row = []
for i in range(data_start_line, len(lines)):
    line = lines[i].strip()
    if not line: # Skip empty lines
        continue

    # Split the numbers by one or more spaces
    current_numbers = [x for x in line.split(' ') if x]

    # Extend the current_row being built
    temp_row.extend(current_numbers)

    # Each logical row has 14 values (13 features + 1 target)
    if len(temp_row) == 14:
        processed_lines.append(" ".join(temp_row)) # Join numbers with space for pd.read_csv
        temp_row = [] # Reset for the next logical row
    elif len(temp_row) > 14: # Handle unexpected extra values if any
        print(f"Warning: Row has more than 14 values. Skipping: {temp_row}")
        temp_row = [] # Reset to avoid cascading errors

# Create a StringIO object from the processed lines
# This simulates a file-like object that pandas can read
data_io = io.StringIO("\n".join(processed_lines))

# Now, read this into a DataFrame
# Use sep=r"\s+" because the numbers are space-separated
# No need for skiprows or header=None since we've pre-processed
data_frame = pd.read_csv(data_io, sep=r"\s+", header=None, names=column_names)

print("DataFrame loaded successfully with all columns:")
data_frame.head()

DataFrame loaded successfully with all columns:


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [227]:
features = data_frame.drop(['INDUS', 'AGE', 'MEDV'], axis=1)
target_values = data_frame['MEDV']

log_prices = np.log(target_values).values
target = pd.DataFrame(log_prices, columns=['PRICE'])

In [217]:
features.mean()

CRIM         3.613524
ZN          11.363636
CHAS         0.069170
NOX          0.554695
RM           6.284634
DIS          3.795043
RAD          9.549407
TAX        408.237154
PTRATIO     18.455534
B          356.674032
LSTAT       12.653063
dtype: float64

In [215]:
features.head()

,CRIM,ZN,CHAS,NOX,RM,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,0,0.538,6.575,4.0900,1,296.0,15.3,396.90,4.98
1,0.02731,0.0,0,0.469,6.421,4.9671,2,242.0,17.8,396.90,9.14
2,0.02729,0.0,0,0.469,7.185,4.9671,2,242.0,17.8,392.83,4.03
3,0.03237,0.0,0,0.458,6.998,6.0622,3,222.0,18.7,394.63,2.94
4,0.06905,0.0,0,0.458,7.147,6.0622,3,222.0,18.7,396.90,5.33


In [253]:
CRIME_IDX = 0
ZN_IDX = 1
CHAS_IDX = 2
RM_IDX = 4
PTRATIO_IDX = 8

property_stats = features.mean().values.reshape(1, 11)

In [235]:
regr = LinearRegression().fit(features, target)
fitted_vals = regr.predict(features)

# Challenge: calculate the MSE and RMSE using sklearn
MSE = mean_squared_error(target, fitted_vals)
RMSE = np.sqrt(MSE)

0.18751213519713034

In [283]:
def get_log_estimate(nr_rooms,
                     students_per_classroom,
                     next_to_river=False,
                     high_confidence=True):
    
    # Configure property
    property_stats[0][RM_IDX] = nr_rooms
    property_stats[0][PTRATIO_IDX] = students_per_classroom

    if next_to_river:
        property_stats[0][CHAS_IDX] = 1
    else:
        property_stats[0][CHAS_IDX] = 0

    # Make prediction
    log_estimate = regr.predict(property_stats)[0][0]

    # Calc Range
    if high_confidence:
        upper_bound = log_estimate + 2*RMSE
        lower_bound = log_estimate - 2*RMSE
        interval = 95
    else:
        upper_bound = log_estimate + RMSE
        lower_bound = log_estimate - RMSE
        interval = 68

    return log_estimate, upper_bound, lower_bound, interval

In [287]:
get_log_estimate(3, 20, next_to_river=True, high_confidence=False)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


(2.7767581914804027, 2.964270326677533, 2.5892460562832724, 68)

In [289]:
np.median(target_values)

21.2

In [299]:
# Challenge: Write the python code that converts the log proce estimate using 1970s prices
# as well as the upper and lower bounds to today's prices. Round the values to the
# nearest 1000 dollars.

ZILLOW_MEDIA_PRICE = 807.9 # ZILLOW_MEDIA_PRICE for boston house values in Jun 2025
SCALE_FACTOR = ZILLOW_MEDIA_PRICE / np.median(target_values)

The estimated property values is 1145000.0
At 68% confidence the valuation range is
USD 949000.0 at the lower to USD 1381000.0 at the high end.


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [329]:
def get_dollar_estimate(rm, ptratio, chas=False, large_range=True):
    """Estimate the price of a property in Boston.

    Keywords arguments:
    rm -- number of rooms in the property.
    ptratio -- number of students per teacher in the classroom for the school in the area.
    chas -- True if the property is next to the river, False otherwhise.
    large_range -- True for a 95% prediction interval, False for a 68% interval.
    
    """

    if rm < 1 or ptratio < 1:
        print('That is unrealistic. Try again.')
        return

    log_est, upper, lower, conf = get_log_estimate(rm,
                                                   students_per_classroom=ptratio,
                                                   next_to_river=chas,
                                                   high_confidence=large_range)

    # Convert to today's dollars
    dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
    dollar_hi = np.e**upper * 1000 * SCALE_FACTOR
    dollar_low = np.e**lower * 1000 * SCALE_FACTOR
    
    # Round the dollar values to nearest thousand
    rounded_east = np.around(dollar_est, -3)
    rounded_hi = np.around(dollar_hi, -3)
    rounded_low = np.around(dollar_low, -3)
    
    print(f'The estimated property values is {rounded_east}')
    print(f'At {conf}% confidence the valuation range is')
    print(f'USD {rounded_low} at the lower to USD {rounded_hi} at the high end.')

In [325]:
get_dollar_estimate(rm=2, ptratio=30, chas=True)

The estimated property values is 385000.0
At 95% confidence the valuation range is
USD 264000.0 at the lower to USD 560000.0 at the high end.


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [337]:
import boston_valuation as val
val.get_dollar_estimate(6, 12, True)

The estimated property values is 1084000.0
At 95% confidence the valuation range is
USD 745000.0 at the lower to USD 1578000.0 at the high end.


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
